In [1]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, EsmForMaskedLM
from tokenizers import Tokenizer
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

/opt/anaconda3/envs/MachLearn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here we run ProGen2 forward and backward taking the average of the logits and attempt to determine the symmetry of its output.


In [2]:
from plm_compare_progen2 import *
from protein_data import *

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model_name = "hugohrban/progen2-medium"
model, tokenizer = initialize_progen2(model_name)

Using cpu device


In [3]:
df_otc_h = pd.read_csv('/Users/johnhutchens/Desktop/Practicum/Data/DMS_ProteinGym_substitutions/OTC_HUMAN_Lo_2023.csv')

In [4]:
df_otc_h.head()

,mutant,mutated_sequence,DMS_score,DMS_score_bin
0,A102E,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.438,1
1,A102G,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.676,1
2,A102P,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.153,0
3,A102S,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.948,1
4,A102T,MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLK...,0.861,1


In [16]:
mut = df_otc_h.iloc[0]['mutant']
seq = df_otc_h.iloc[0]['mutated_sequence']
print(mut)
print(seq)

A102E
MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGKSLGMIFEKRSTRTRLSTETGFELLGGHPCFLTTQDIHLGVNESLTDTARVLSSMADAVLARVYKQSDLDTLAKEASIPIINGLSDLYHPIQILADYLTLQEHYSSLKGLTLSWIGDGNNILHSIMMSAAKFGMHLQAATPKGYEPDASVTKLAEQYAKENGTKLLLTNDPLEAAHGGNVLITDTWISMGQEEEKKKRLQAFQGYQVTMKTAKVAASDWTFLHCLPRKPEEVDDEVFYSPRSLVFPEAENRKWTIMAVMVSLLTDYSPQLQKPKF


In [34]:
len_mut = len(mut)
orig = mut[0]
pos = int(mut[1:len_mut-1])-1
new = mut[len_mut-1]

In [35]:
wild_seq = seq[:pos] + orig + seq[pos+1:]

In [38]:
print(seq[101])
wild_seq[101]

E


'A'

In [23]:
with open('/Users/johnhutchens/Desktop/Practicum/Data/OTC_Human/pg2_otc_hum_matrices.pickle',
           'rb') as f:
    pg_dict = pickle.load(f)

In [39]:
pg_dict[None] = {'mutated_sequence': wild_seq}

In [40]:
pg_dict[None]

{'mutated_sequence': 'MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGKSLGMIFEKRSTRTRLSTETGFALLGGHPCFLTTQDIHLGVNESLTDTARVLSSMADAVLARVYKQSDLDTLAKEASIPIINGLSDLYHPIQILADYLTLQEHYSSLKGLTLSWIGDGNNILHSIMMSAAKFGMHLQAATPKGYEPDASVTKLAEQYAKENGTKLLLTNDPLEAAHGGNVLITDTWISMGQEEEKKKRLQAFQGYQVTMKTAKVAASDWTFLHCLPRKPEEVDDEVFYSPRSLVFPEAENRKWTIMAVMVSLLTDYSPQLQKPKF'}